# ALD Study

In [ ]:
from pathlib import Path
import pandas as pd
import vaep

In [ ]:
file = 'data/applications/ald_proteome_spectronaut.tsv'

In [ ]:
data = pd.read_table(file, low_memory=False)
data.shape

In [ ]:
data

In [ ]:
data.iloc[:,:8].describe(include='all')

In [ ]:
column_types = data.iloc[:,8:].columns.to_series().apply(lambda s: tuple(s.split('.')[-2:]))
column_types.describe() ## .apply(lambda l: l[-1])

In [ ]:
column_types = ['.'.join(x for x in tup) for tup in list(column_types.unique())]
column_types

In [ ]:
data = data.set_index(list(data.columns[:8])).sort_index(axis=1)

In [ ]:
data.loc[:,data.columns.str.contains(column_types[0])]

In [ ]:
data.iloc[:20,:6]

create new multiindex from column

In [ ]:
data.columns = pd.MultiIndex.from_tuples(data.columns.str.split().str[1].str.split('.raw.').to_series().apply(tuple), names = ['Sample ID', 'vars'])
data = data.stack(0)
data

## Meta data

In [ ]:
meta = data.index.to_frame().reset_index(drop=True)
meta

In [ ]:
meta.describe()

## Data and metadata

taken from [Spectronaut manuel](https://biognosys.com/resources/spectronaut-manual/)

feature | description 
--- | ---
PEP.IsProteinGroupSpecific | True or False. Tells you whether the peptide only belongs to one Protein Group.
PEP.StrippedSequence | -
PEP.IsProteotypic |  -
PEP.PeptidePosition | -
PG.Cscore | - 
PG.ProteinAccessions | -
PG.Genes | - 
PEP.Quantity | The quantitative value for that peptide as defined in the settings.
EG.PrecursorId | Unique Id for the precursor: [modified sequence] plus [charge] 
EG.Qvalue | The q-value (FDR) of the EG.
EG.TotalQuantity (Settings) | The quantitative value for that EG as defined in the settings. 

> Headers related to Peptides (PEP) as defined in the settings. Many headers related to Peptides are self-explanatory. Here are the most relevant and some which are not too obvious. 

> Headers related to Peptides (PEP) as defined in the settings. Many headers related to Peptides are self-explanatory. Here are the most relevant and some which are not too obvious. 

After discussing with Lili, `PEP.Quantity` is the fitting entity for each unique aggregated Peptide. Duplicated entries are just to drop

In [ ]:
sel_cols = ['Sample ID', 'PEP.StrippedSequence', 'PEP.Quantity']
sel_data = data.reset_index()[sel_cols].drop_duplicates().set_index(sel_cols[:2])
sel_data

In [ ]:
sel_data = sel_data.squeeze().dropna().astype(float).unstack()
sel_data

In [ ]:
idx = sel_data.index.to_series()
idx = idx.str.extract(r'(Plate[\d]_[A-H]\d*)').squeeze()
idx.name = 'Sample ID'
idx.describe()

In [ ]:
sel_data = sel_data.set_index(idx)
sel_data.to_pickle('data/single_datasets/ald_aggPeptides_spectronaut.pkl')